In [1]:
from MFEA_lib.model import Firework,MFEA1,SMP_MFEA
from MFEA_lib.model.utils import *
from MFEA_lib.operators.Crossover import SBX_Crossover,newSBX
from MFEA_lib.operators.Mutation import Polynomial_Mutation
from MFEA_lib.operators.Selection import ElitismSelection
from MFEA_lib.operators.Search import L_SHADE
from MFEA_lib.tasks.Benchmark.Funcs import CEC17_benchmark
from MFEA_lib.tasks.Benchmark.IDPC_EDU import IDPC_EDU_benchmark_1,IDPC_EDU_benchmark_2
from MFEA_lib.EA import *

In [2]:
tasks = IDPC_EDU_benchmark_2.get_tasks(1)
print('num tasks: ', len(tasks))

Reading data...


100%|██████████| 10/10 [00:00<00:00, 50.39it/s]

num tasks:  10


In [3]:
import numpy as np
x = np.random.rand(tasks[0].dim)
tasks[0](x)

99999

In [4]:
class IndClass(Individual):
    def __init__(self, genes, parent= None, dim= None, *args, **kwargs) -> None:
        super().__init__(genes, parent, dim)
        if genes is None:
            self.genes: np.ndarray = np.random.rand(dim)
        self.A = np.ones(len(self.genes))

In [16]:
baseModel = MFEA1.model()
baseModel.compile(
    IndClass= IndClass,
    tasks= tasks,
    crossover= SBX_Crossover(nc = 2),
    mutation= Polynomial_Mutation(nm = 5),
    selection= ElitismSelection()
)
baseModel.fit(
    nb_generations = 1000, rmp = 0.3, nb_inds_each_task= 100, 
    bound_pop= [0, 1], evaluate_initial_skillFactor= True
)

Time: 03m 7.60s  100 % [====================>]  Cost: 7.00E+00  1.20E+01  1.20E+01  1.60E+01  1.60E+01  1.80E+01  2.40E+01  2.10E+01  2.10E+01  2.80E+01  ,  
END!


[Genes: [1.         0.         1.         0.         0.         1.
  0.8686664  0.25327265 0.2991757  0.68616777 0.80986102 0.24584145
  0.09703512 0.71729067 0.01679808 0.68566648 0.41685236 0.74465091
  0.62811275 0.95052358 0.59481196 0.17923758 0.72722168 0.2662808
  1.        ]
 Skill_factor: 0,
 Genes: [0.15056929 0.00117425 0.85970604 0.60014299 0.77125542 0.7642627
  0.57518088 0.54987439 0.28116855 0.27386587 0.66503261 0.15841931
  0.73234031 0.2231207  0.20638864 0.69216062 0.36605084 0.64085495
  0.74415895 0.54339425 0.41477318 0.67631854 0.13531981 0.48823142
  0.73875974]
 Skill_factor: 1,
 Genes: [0.08636899 0.50735449 0.46785664 0.84743148 0.10262001 0.67640799
  0.61139987 0.06317533 0.57640315 0.34538074 0.08824296 0.95854064
  0.11018659 0.90613378 0.70934892 0.23283145 0.57968902 0.78202828
  0.0404815  0.52768389 0.31904515 0.56947897 0.92525934 0.70950681
  0.05096163]
 Skill_factor: 2,
 Genes: [0.68050268 0.12557352 0.33592739 0.45961262 0.3745596  0.66480557
  

In [6]:
tasks[0].num_edges

990

In [11]:
gene = baseModel.last_pop[0].ls_inds
gene

[Genes: [0.22675021 0.00471385 0.         0.76140181 1.         0.88429165
  0.52887575 0.55018995 0.66763392 0.12681608 0.99951871 0.63841679
  0.70348981 0.         0.07443993 0.15589086 0.62638873 0.
  0.9926335  0.56833913 0.32981419 0.78991378 0.6706972  0.08375581
  0.36563949]
 Skill_factor: 0,
 Genes: [0.         0.         0.00388868 1.         0.00912775 1.
  0.99971913 0.12642041 0.97191595 0.75649164 0.85055216 0.18499792
  1.         1.         0.01933849 0.73252562 0.27854144 0.01218849
  0.65321084 0.67596179 0.40913762 0.37378199 0.85438764 1.
  0.46138138]
 Skill_factor: 0,
 Genes: [0.00121182 0.         0.00307324 1.         0.         1.
  0.93677384 0.28729323 0.95879198 0.89750237 0.84658334 0.2473352
  0.97623403 1.         0.03920862 0.82064318 0.02982223 0.01313001
  0.65717891 0.67562024 0.41797    0.38254956 0.59775824 0.93723681
  0.47007029]
 Skill_factor: 0,
 Genes: [0.         0.23195645 1.         1.         0.57982009 0.1100763
  0.5945763  0.06991434 0.

In [15]:
gene_list = [baseModel.last_pop[0].ls_inds[i].genes for i in range(len(baseModel.last_pop[0].ls_inds))]
for gene in gene_list:
    cost =0 
    leftDomain = -1
    domain = np.zeros(tasks[0].num_domains+1)
    nodes = np.zeros(tasks[0].num_nodes+1)
    currNode=  tasks[0].source
    for idx in range(len(gene)):
        if currNode == tasks[0].target:
            print(cost)
            break
        l = int(gene[idx]*len(tasks[0].edges[currNode]))
        if l == len(tasks[0].edges[currNode]):
            l = len(tasks[0].edges[currNode])-1
        # print(len(tasks[0].edges[currNode]))
        # print(tasks[0].edges[currNode])
        nextNode,w,d = tasks[0].edges[currNode][l]
        nodes[nextNode] = 1
        domain[d] = 1
        leftDomain = d
        currNode = nextNode
        cost += w
    break

1
8
7
13


In [9]:
tasks[0].edges

{1: [[6, 2, 2],
  [4, 1, 6],
  [3, 1, 4],
  [5, 1, 2],
  [9, 12, 5],
  [2, 27, 3],
  [8, 28, 7],
  [5, 16, 3],
  [5, 16, 4],
  [4, 17, 4],
  [6, 25, 4],
  [2, 22, 6],
  [4, 28, 1],
  [5, 23, 1],
  [7, 28, 1],
  [6, 25, 10],
  [3, 27, 3],
  [4, 12, 10],
  [8, 15, 6],
  [4, 21, 8],
  [7, 14, 1],
  [8, 11, 10],
  [4, 13, 2],
  [2, 13, 4],
  [7, 17, 7],
  [2, 18, 5],
  [2, 19, 7],
  [7, 23, 2],
  [9, 11, 3],
  [4, 27, 3],
  [4, 22, 8],
  [6, 25, 3],
  [2, 20, 8],
  [2, 22, 2],
  [2, 14, 3],
  [5, 17, 3],
  [5, 21, 3],
  [2, 23, 5],
  [9, 17, 5],
  [6, 27, 6],
  [2, 17, 4],
  [9, 21, 7],
  [3, 15, 5],
  [8, 21, 8],
  [6, 14, 3],
  [6, 13, 7],
  [5, 25, 7],
  [7, 22, 4],
  [5, 22, 7],
  [7, 16, 5],
  [7, 16, 9],
  [6, 16, 10],
  [7, 20, 1],
  [9, 17, 5],
  [7, 27, 3],
  [5, 19, 6],
  [5, 16, 6],
  [7, 15, 10],
  [8, 18, 7],
  [4, 16, 5],
  [3, 19, 7],
  [3, 22, 7],
  [9, 19, 3],
  [2, 25, 1],
  [7, 13, 7],
  [3, 29, 5],
  [3, 14, 5],
  [2, 12, 7],
  [6, 28, 9],
  [3, 24, 5],
  [5, 13, 10],
 